#### ![Roboflow](https://media-exp1.licdn.com/dms/image/C4D1BAQEf8_Y4I1CJwA/company-background_10000/0/1631290065486?e=2159024400&v=beta&t=0RCuQDNh2uELCij-75lsTMJFgCcCzveJh7fOJNaCW3E)

# Configure remote GSC storage

In [ ]:
project_name = "lucky-charms-detection"

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import uuid
bucket = 'colab-notebook-' + str(uuid.uuid1())

In [ ]:
!gcloud config set project {project_name}

In [ ]:
!gsutil mb gs://{bucket}

# Downloads and Installations

## Download and install the TensorFlow Object Detection API

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%%capture
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

## Download pretrained model

In [ ]:
model = "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"

In [ ]:
model_config = model #"ssd_efficientdet_d4_1024x1024_coco17_tpu-32"

In [ ]:
%cd /content
!wget -O model.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/{model}.tar.gz
!tar -xvf model.tar.gz

In [ ]:
!wget -O model.config https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/{model_config}.config

## Download annotated dataset from Roboflow as TFRecord

In [ ]:
%%bash
mkdir /content/dataset
cd /content/dataset

curl -L "https://app.roboflow.com/ds/QuKgeWgrhO?key=nYPM5QxVU2" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip #raw dataset
#curl -L "https://app.roboflow.com/ds/gMetmOhec6?key=JSiBw1ByMW" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip ///55x augmentations

# Setup

## Setup traning pipeline for our file locations

In [ ]:
pipeline_config_path = 'model.config'

model_dir = 'gs://{0}/training'.format(bucket)
fine_tune_checkpoint = 'gs://{0}/{1}/checkpoint/ckpt-0'.format(bucket, model)

label_map_path = 'gs://{0}/dataset/train/marbits_label_map.pbtxt'.format(bucket)
train_record_path = 'gs://{0}/dataset/train/marbits.tfrecord'.format(bucket)
test_record_path = 'gs://{0}/dataset/test/marbits.tfrecord'.format(bucket)

## Setup training variables

In [ ]:
num_steps = 10000
batch_size = 32

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes('/content/dataset/test/marbits_label_map.pbtxt')

## Configure pipeline configuration file with our specific file locations and training variables

In [ ]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
  'label_map_path: "{}"'.format(label_map_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint),
                  config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set training batch_size.
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set fine tune checkpoint type to 'detection'.
  config = re.sub('(fine_tune_checkpoint_type: ".*?)(classification)(.*?")', 
                  'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  
  f.write(config)

In [ ]:
pipeline_config_path = 'gs://{0}/model.config'.format(bucket)

## Upload to Google Cloud Storage

In [ ]:
%cd /content
!gsutil cp -r dataset/ gs://{bucket}
!gsutil cp -r {model}/ gs://{bucket}
!gsutil cp model.config gs://{bucket}

# Train model

## Install OpenCV Python Headless

*Due to a current issue with Colab's Version of OpenCV Python, you must manually install in order for the code to run*

In [ ]:
!pip install "opencv-python-headless<4.3"

## Run training

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --use_tpu=True \
    --alsologtostderr

# Export model

## Save model checkpoints as frozen graph

In [ ]:
export_dir = '/content/export/'

In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path {pipeline_config_path} \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {export_dir}

## Save model to GCS

In [ ]:
!gsutil cp -r export/ gs://{bucket}